In [20]:
import pandas as pd
import os
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\User\Downloads\Projeto2_CDados


In [21]:
dados = pd.read_csv('drinks.csv')
temp = pd.read_excel('temp.xlsx')
dados

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
0,Afghanistan,0,0,0,0.0
1,Albania,89,132,54,4.9
2,Algeria,25,0,14,0.7
3,Andorra,245,138,312,12.4
4,Angola,217,57,45,5.9
5,Antigua & Barbuda,102,128,45,4.9
6,Argentina,193,25,221,8.3
7,Armenia,21,179,11,3.8
8,Australia,261,72,212,10.4
9,Austria,279,75,191,9.7


In [22]:
mergedStuff = pd.merge(dados, temp, on=['country'], how='left')
drop_reps = mergedStuff.drop_duplicates(subset=None, keep='first', inplace=False)
final = drop_reps.dropna(subset = ['temperature'], inplace = False)
final

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,temperature
0,Afghanistan,0,0,0,0.0,16.0
1,Albania,89,132,54,4.9,14.0
2,Algeria,25,0,14,0.7,25.0
3,Andorra,245,138,312,12.4,11.0
4,Angola,217,57,45,5.9,23.0
6,Argentina,193,25,221,8.3,15.0
7,Armenia,21,179,11,3.8,12.0
8,Australia,261,72,212,10.4,22.0
9,Austria,279,75,191,9.7,7.0
10,Azerbaijan,21,46,5,1.3,14.0


In [23]:
final.describe()


,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,temperature
count,170.000000,170.000000,170.000000,170.000000,170.000000
mean,109.770588,79.141176,52.370588,4.831176,19.605882
std,103.111592,85.355456,83.492254,3.815823,8.677180
min,0.000000,0.000000,0.000000,0.000000,-15.000000
25%,20.250000,5.000000,1.000000,1.300000,12.250000
50%,77.000000,60.500000,8.000000,4.500000,23.000000
75%,192.750000,122.000000,61.250000,7.275000,27.000000
max,376.000000,438.000000,370.000000,14.400000,30.000000


In [26]:
final.dtypes

country                          object
beer_servings                     int64
spirit_servings                   int64
wine_servings                     int64
total_litres_of_pure_alcohol    float64
temperature                     float64
dtype: object

In [28]:
correlacao = final.corr()
correlacao

,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,temperature
beer_servings,1.000000,0.441931,0.531823,0.831776,-0.448889
spirit_servings,0.441931,1.000000,0.189116,0.632028,-0.312126
wine_servings,0.531823,0.189116,1.000000,0.676209,-0.549436
total_litres_of_pure_alcohol,0.831776,0.632028,0.676209,1.000000,-0.537148
temperature,-0.448889,-0.312126,-0.549436,-0.537148,1.000000


In [30]:
#final.hist()